Evaluate the ability of the model to pick questions relevant to the topic on CAsT 19 test and Tasks 15. Question sequences are considered as sets here.

In [7]:
test_on = 'tasks15'  # cast19_test or tasks15

In [8]:
# load gt topics
from collections import defaultdict, Counter

if test_on == 'cast19_test':
    path_qs = '../data/cast19/test_questions.tsv'
    path_index = "/home/svakule/markers_bert/Anserini/indexes/anserini_index_gt_cast19_test"
elif test_on == 'tasks15':
    path_qs = '../data/tasks15/test_questions.tsv'
    path_index = "/home/svakule/markers_bert/Anserini/indexes/tasks15_gt"
    
question_topic = {}
t_qs = defaultdict(list)
with open(path_qs, 'r') as f:
    for line in f:
        q_id, q = line.strip('\n').split('\t')
        topic = q_id.split('_')[0]
        t_qs[topic].append(q)
        question_topic[q] = topic

print("Total questions: %d in %s" % (len(question_topic), test_on))

Total questions: 547 in tasks15


# BM25-1

In [9]:
# setup index search for similar generated questions
import json
from pyserini.search import SimpleSearcher

def search_similar_question(query_str, searcher, history, num_candidates_samples):
    # sample next questions by similarity
    hits = searcher.search(query_str, k=num_candidates_samples)
    for j, hit in enumerate (hits):
        q = json.loads(hit.raw)['contents']
        # make sure we are not asking the same question
        if q not in history:
            return q
    return None


num_errors = 0
nones = 0
evaluated_questions = 0

searcher = SimpleSearcher(path_index)
for topic, qs in t_qs.items():
    history = []
    for i, q in enumerate(qs[:-1]):
        history.append(q)
        print('GT question:', q)
        evaluated_questions += 1
        
        # sample next question
        query_str = search_similar_question(q, searcher, history, num_candidates_samples=10000)
        
        # check set
        if query_str not in qs:
            num_errors += 1
            print('Error sampled question:', query_str)
            if not query_str:
                nones += 1
        else:
            print('Correct sampled question:', query_str)

acc = ((evaluated_questions - num_errors) / evaluated_questions) * 100
frac_nones = (nones / evaluated_questions) * 100
print("Accuracy %.2f None %.2f Qs %d" % (acc, frac_nones, evaluated_questions))

GT question: getting organized at work
Correct sampled question: Checklist for getting organized at work
GT question: Checklist for getting organized at work
Correct sampled question: Tips for getting organized at work
GT question: How to organize office desk
Correct sampled question: Organize schedule at office
GT question: Tips for getting organized at work
Correct sampled question: How to organize your work space
GT question: Organize schedule at office
Correct sampled question: How to keep a calendar of scheduled meetings and travel
GT question: How to create a todo/task list
Error sampled question: Create a tropical area inside a house and learn how to care for indoor plants
GT question: How to keep a calendar of scheduled meetings and travel
Error sampled question: How to keep trees and shrubs to be safe during hurricane
GT question: How to set deadlines and goals
Correct sampled question: How to set up a filing system with a binder or folders
GT question: How to organize your wo

Correct sampled question: investigate living conditions in India
GT question: investigate living conditions in India
Correct sampled question: look for other people similar to me living in India
GT question: find a place to live
Error sampled question: Look up best/worst places to live for Fibromyalgia patients
GT question: look for entertainment/things to do
Error sampled question: Find out about things to do while in Barbados
GT question: look for current news about India
Correct sampled question: look for other people similar to me living in India
GT question: build raised gardens
Correct sampled question: build raised garden beds
GT question: find a good place for the garden
Error sampled question: find a place to live
GT question: acquire materials (tools, boards/frame, soil, etc)
Correct sampled question: fertilize soil and plant
GT question: build raised garden beds
Error sampled question: Prepare your garden for the winter
GT question: build a frame
Correct sampled question: in

Error sampled question: Find ways to prevent cervical cancer
GT question: Buy a blood oxygen/pulse meter
Error sampled question: Install extra power meters
GT question: Check where the emergency rooms/hospitals are
Error sampled question: Check the weather
GT question: Appropriate medicine for altitude sickness
Error sampled question: Take appropriate medicine (ex. tetanus booster shot or an anti venom)
GT question: Ask a doctor's advice
Error sampled question: Ask an agriculturist advice
GT question: Find a hotel with oxygen supplemented rooms
Error sampled question: Book a hotel
GT question: Find out what type of food and beverages are appropriate to consume
Error sampled question: Find out the appropriate height to install the detector
GT question: septic system options
Correct sampled question: What are the different septic system types (e.g. gravity, pressure distribution, aerobic treatment, sand filter, etc)?
GT question: What are the different septic system types (e.g. gravity, 

Error sampled question: Find (trusted) auction houses
GT question: live with fibromyalgia
Correct sampled question: Learn how to manage family issues while living with Fibromyalgia
GT question: Find out about support groups
Correct sampled question: Find out about different chat rooms for support
GT question: Learn how to cope with pain
Correct sampled question: Find how to cope sustain a normal life with symptoms
GT question: Find best blogs and articles on Fibromyalgia
Correct sampled question: Look up best/worst places to live for Fibromyalgia patients
GT question: Find how to cope sustain a normal life with symptoms
Correct sampled question: Find first person narratives on creating a good life with the disease
GT question: Find out about different chat rooms for support
Error sampled question: Find out how to get government support
GT question: Learn how to manage family issues while living with Fibromyalgia
Error sampled question: Learn how to place a bid while buying condo
GT que

Error sampled question: None
GT question: Put the greenbacks into green efficiency
Correct sampled question: Add energy efficient windows
GT question: Upgrade your bath
Correct sampled question: Remodel your kitchen
GT question: Remodel your kitchen
Error sampled question: Clean your furnace
GT question: Add energy efficient windows
Error sampled question: Add weather stripping to doors and windows
GT question: Roof replacement
Error sampled question: How to find a leak in roof
GT question: Refurbish the front door
Error sampled question: Caulk cracks around windows and doors
GT question: deliver effective tech presentations
Correct sampled question: Sample effective tech presentations
GT question: Find courses for delivering presentations
Correct sampled question: How to handle stage fear while delivering presentations
GT question: Learn how to prepare a presentation deck (e.g. use visual aids)
Error sampled question: Prepare presentation deck to send to VC
GT question: Find technical

Error sampled question: Ways of non medicine treatment (ex. Wash the site of the spider bite well with soap and water, apply a cool compress,etc)
GT question: Get a service dog
Error sampled question: Choose a hosting service
GT question: Learn methods for effective teaching (delineate the teaching space, predictable schedule, identify strategies for proper information absorption)
Correct sampled question: Handling a lack of responsiveness (teach social skills, encourage social interactions)
GT question: Learn methods to communicate (picture communication systems, object labelling, sign language)
Error sampled question: Get parenting help (e.g. parenting classes or single mothers (online) communities)
GT question: Choose toys to stimulate their senses, develop social interactions, motor and organization skills
Correct sampled question: Handling a lack of responsiveness (teach social skills, encourage social interactions)
GT question: Regular health checks
Error sampled question: Learn 

# BM25-n

In [66]:
# setup index search for similar generated questions
import json
from pyserini.search import SimpleSearcher

def search_similar_question(query_str, searcher, history, num_candidates_samples, n_history=6):
    history_str2 = ' '.join(history[-n_history:])
#     print("%d previous questions: %s" % (n_history, history_str2))
    # sample next questions by similarity
    hits = searcher.search(history_str2, k=num_candidates_samples)
    for j, hit in enumerate (hits):
        q = json.loads(hit.raw)['contents']
        # make sure we are not asking the same question
        if q not in history:
            return q
    return None


num_errors = 0
nones = 0
evaluated_questions = 0

searcher = SimpleSearcher(path_index)
for topic, qs in t_qs.items():
    history = []
    for i, q in enumerate(qs[:-1]):
        history.append(q)
#         print('GT question:', q)
        evaluated_questions += 1
        
        # sample next question
        query_str = search_similar_question(q, searcher, history, num_candidates_samples=10000)
        
        # check set
        if query_str not in qs:
            num_errors += 1
#             print('Error sampled question:', query_str)
            if not query_str:
                nones += 1
#         else:
#             print('Correct sampled question:', query_str)

acc = ((evaluated_questions - num_errors) / evaluated_questions) * 100
frac_nones = (nones / evaluated_questions) * 100
print("Accuracy %.2f None %.2f Qs %d" % (acc, frac_nones, evaluated_questions))

Accuracy 68.81 None 0.00 Qs 497


# BM25-n +BERT-n

In [10]:
# load pre-trained model
import numpy as np
from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# output_dir = './question_sampler_model/'
# output_dir = './question_sampler_model_bert_history/'
output_dir = './question_sampler_model_bert_history_long/'
# output_dir = './qrecc_question_sampler_model_bert_history_long_2epochs/'



tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(output_dir)

def infer(q1, q2, tokenizer, model):
    inputs = tokenizer(q1, q2, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    preds = logits.detach().cpu().numpy()
    pred_labels_i = np.argmax(preds, axis=1).flatten()
    return int(pred_labels_i[0])

q1 = 'What are the different types of tachycardia?'
q2 = 'What are the types of lipids?'
print(infer(q1, q2, tokenizer, model))


q1 = 'Should I brine a turkey before smoking it?'
q2 = 'How long should the turkey be smoked for?'
infer(q1, q2, tokenizer, model)

0


1

In [11]:
# setup index search for similar generated questions
import json
from pyserini.search import SimpleSearcher

def search_similar_question(query_str, searcher, history, num_candidates_samples, n_history=6):
    history_str = ' '.join(history[-n_history:])
#     print("%d previous questions: %s" % (n_history, history_str))
    # sample next questions by similarity
    hits = searcher.search(history_str, k=num_candidates_samples)
    for j, hit in enumerate (hits):
        q = json.loads(hit.raw)['contents']
        # make sure we are not asking the same question
        if q not in history:
            # model predict if the question is on topic
            if infer(history_str, q, tokenizer, model):
                return q
    return None


num_errors = 0
nones = 0
evaluated_questions = 0

searcher = SimpleSearcher(path_index)
for topic, qs in t_qs.items():
    history = []
    for i, q in enumerate(qs[:-1]):
        history.append(q)
#         print('GT question:', q)
        evaluated_questions += 1
        
        # sample next question
        query_str = search_similar_question(q, searcher, history, num_candidates_samples=10000)
        
        # check set
        if query_str not in qs:
            num_errors += 1
            print('Error sampled question:', query_str)
            if not query_str:
                nones += 1
#         else:
#             print('Correct sampled question:', query_str)

acc = ((evaluated_questions - num_errors) / evaluated_questions) * 100
frac_nones = (nones / evaluated_questions) * 100
print("Accuracy %.2f None %.2f #Qs %d #Errors %d" % (acc, frac_nones, evaluated_questions, num_errors))

Error sampled question: How to prepare for job interview
Error sampled question: How to prepare for job interview
Error sampled question: How to prepare for job interview
Error sampled question: Review and compare decoration options
Error sampled question: Review and compare decoration options
Error sampled question: What to consider when deciding on the wedding budget
Error sampled question: Choose mirrors
Error sampled question: None
Error sampled question: Find and book suitable transportation
Error sampled question: Find and book suitable transportation
Error sampled question: Find and book suitable transportation
Error sampled question: install gate
Error sampled question: install gate
Error sampled question: Search how to connect it to AC-powered connectors
Error sampled question: build a frame
Error sampled question: What is the cost of the materials?
Error sampled question: acquire materials (tools, boards/frame, soil, etc)
Error sampled question: acquire materials (tools, boar

In [50]:
# setup index search for similar generated questions
import json
from pyserini.search import SimpleSearcher

def search_similar_question(query_str, searcher, history, num_candidates_samples, n_history=7):
    history_str = ' '.join(history[-n_history:])
    print("%d previous questions: %s" % (n_history, history_str))
    # sample next questions by similarity
    hits = searcher.search(history_str, k=num_candidates_samples)
    for j, hit in enumerate (hits):
        q = json.loads(hit.raw)['contents']
        # make sure we are not asking the same question
        if q not in history:
            # model predict if the question is on topic
            if infer(' '.join(history[-2:]), q, tokenizer, model):
                return q
    return None


num_errors = 0
nones = 0
evaluated_questions = 0

searcher = SimpleSearcher(path_index)
for topic, qs in t_qs.items():
    history = []
    for i, q in enumerate(qs[:-1]):
        history.append(q)
#         print('GT question:', q)
        evaluated_questions += 1
        
        # sample next question
        query_str = search_similar_question(q, searcher, history, num_candidates_samples=10000)
        
        # check set
        if query_str not in qs:
            num_errors += 1
            print('GT question:', q)
            print('Error sampled question:', query_str)
            if not query_str:
                nones += 1
#         else:
#             print('Correct sampled question:', query_str)

acc = ((evaluated_questions - num_errors) / evaluated_questions) * 100
frac_nones = (nones / evaluated_questions) * 100
print("Accuracy %.2f None %.2f #Qs %d #Errors %d" % (acc, frac_nones, evaluated_questions, num_errors))

7 previous questions: What is throat cancer?
7 previous questions: What is throat cancer? Is throat cancer treatable?
7 previous questions: What is throat cancer? Is throat cancer treatable? Tell me about lung cancer.
7 previous questions: What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer's symptoms?
7 previous questions: What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer's symptoms? Can lung cancer spread to the throat?
7 previous questions: What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer's symptoms? Can lung cancer spread to the throat? What causes throat cancer?
7 previous questions: What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer's symptoms? Can lung cancer spread to the throat? What causes throat cancer? What is the first sign of throat cancer?
7 previous questions: Is throat cancer tr

7 previous questions: Tell me about the history of toilets. Where does the term toilet come from? Where and when was the first toilet invented? Why do the Brits call a toilet a loo?
7 previous questions: Tell me about the history of toilets. Where does the term toilet come from? Where and when was the first toilet invented? Why do the Brits call a toilet a loo? What came before toilets?
7 previous questions: Tell me about the history of toilets. Where does the term toilet come from? Where and when was the first toilet invented? Why do the Brits call a toilet a loo? What came before toilets? When did indoor versions of toilets become common?
7 previous questions: Tell me about the history of toilets. Where does the term toilet come from? Where and when was the first toilet invented? Why do the Brits call a toilet a loo? What came before toilets? When did indoor versions of toilets become common? How many types of toilets are there?
7 previous questions: Where does the term toilet come f

7 previous questions: What does it mean to be a vegan? What is the environmental argument for being a vegan? Do you need milk for strong bones? What are other good sources of calcium besides milk?
7 previous questions: What does it mean to be a vegan? What is the environmental argument for being a vegan? Do you need milk for strong bones? What are other good sources of calcium besides milk? If you don't eat any meat, at all, is it bad for you?
7 previous questions: What does it mean to be a vegan? What is the environmental argument for being a vegan? Do you need milk for strong bones? What are other good sources of calcium besides milk? If you don't eat any meat, at all, is it bad for you? What are good sources of vitamin B12?
7 previous questions: What does it mean to be a vegan? What is the environmental argument for being a vegan? Do you need milk for strong bones? What are other good sources of calcium besides milk? If you don't eat any meat, at all, is it bad for you? What are goo

7 previous questions: What are some interesting things around Ann Arbor? Tell me about when the city of Ann Arbor was founded. How does the founding of the city of Ann Arbor relate to the establishment of the University? Does the Museum of Art in Ann Arbor have any special collections? What is the South Pond Nature Area in Ann Arbor? Are there any film festivals in Ann Arbor?
7 previous questions: What are some interesting things around Ann Arbor? Tell me about when the city of Ann Arbor was founded. How does the founding of the city of Ann Arbor relate to the establishment of the University? Does the Museum of Art in Ann Arbor have any special collections? What is the South Pond Nature Area in Ann Arbor? Are there any film festivals in Ann Arbor? How about for jazz festivals in Ann Arbor?
7 previous questions: What causes acidic reflux in the morning?
7 previous questions: What causes acidic reflux in the morning? Does acidic reflux in the morning have long term side effects?
7 previo

7 previous questions: How does the oceanic crust differ from the continental crust? Which of the oceanic crust and the continental crust is younger and why? Where is the youngest oceanic crust found? Where is the oldest oceanic crust found? What is seafloor spreading? What is the significance of seafloor spreading? How is ocean crust formed?
7 previous questions: What is Darwin’s theory in a nutshell?
7 previous questions: What is Darwin’s theory in a nutshell? How was Darwin’s theory developed?
7 previous questions: What is Darwin’s theory in a nutshell? How was Darwin’s theory developed? How do sexual and asexual reproduction affect Darwin’s theory?
7 previous questions: What is Darwin’s theory in a nutshell? How was Darwin’s theory developed? How do sexual and asexual reproduction affect Darwin’s theory? How can fossils be used to understand Darwin’s theory?
7 previous questions: What is Darwin’s theory in a nutshell? How was Darwin’s theory developed? How do sexual and asexual repr

7 previous questions: Who are The Avengers? Tell me about The Avengers' first appearance. Who is the most powerful of The Avengers and why? What is the relationship of Spider-Man to The Avengers? Why is Batman not a member of The Avengers? What is an important team in the DC universe?
7 previous questions: Who are The Avengers? Tell me about The Avengers' first appearance. Who is the most powerful of The Avengers and why? What is the relationship of Spider-Man to The Avengers? Why is Batman not a member of The Avengers? What is an important team in the DC universe? Tell me about the origins of the Justice League in the DC universe.
7 previous questions: Tell me about The Avengers' first appearance. Who is the most powerful of The Avengers and why? What is the relationship of Spider-Man to The Avengers? Why is Batman not a member of The Avengers? What is an important team in the DC universe? Tell me about the origins of the Justice League in the DC universe. Who are the important member

7 previous questions: What is nominal GDP? Why is nominal GDP not a good measure? What is the between nominal and real GDP? How is nominal GDP related to gross national income? What is the meaning of GNI? How is inflation affected by high GDP growth? What is the relationship between inflation and unemployment?
7 previous questions: Why is nominal GDP not a good measure? What is the between nominal and real GDP? How is nominal GDP related to gross national income? What is the meaning of GNI? How is inflation affected by high GDP growth? What is the relationship between inflation and unemployment? What is frictional unemployment and why is it important?
7 previous questions: What is the between nominal and real GDP? How is nominal GDP related to gross national income? What is the meaning of GNI? How is inflation affected by high GDP growth? What is the relationship between inflation and unemployment? What is frictional unemployment and why is it important? Is frictional unemployment bad 

7 previous questions: How was melatonin discovered? What are good sources of melatonin in food? Is melatonin bad for you? What are the side effects of melatonin? Why does melatonin require a prescription in the UK? How can I increase my melatonin levels naturally? Is melatonin effective for treating insomnia?
7 previous questions: What is solar energy?
7 previous questions: What is solar energy? What are the main uses of solar energy?
7 previous questions: What is solar energy? What are the main uses of solar energy? Why is solar energy important?
7 previous questions: What is solar energy? What are the main uses of solar energy? Why is solar energy important? What are advantages of solar energy?
7 previous questions: What is solar energy? What are the main uses of solar energy? Why is solar energy important? What are advantages of solar energy? What is the cost of solar power?
7 previous questions: What is solar energy? What are the main uses of solar energy? Why is solar energy impor

7 previous questions: What are the pros and cons of electric cars? Tell me more about Tesla the car company. Why did Elon Musk start Tesla? What Tesla car models are available?
7 previous questions: What are the pros and cons of electric cars? Tell me more about Tesla the car company. Why did Elon Musk start Tesla? What Tesla car models are available? What is the best selling Tesla car model?
7 previous questions: What are the pros and cons of electric cars? Tell me more about Tesla the car company. Why did Elon Musk start Tesla? What Tesla car models are available? What is the best selling Tesla car model? What is unique about the Tesla Model 3?
7 previous questions: What are the pros and cons of electric cars? Tell me more about Tesla the car company. Why did Elon Musk start Tesla? What Tesla car models are available? What is the best selling Tesla car model? What is unique about the Tesla Model 3? What are the safety features of Tesla Model 3?
7 previous questions: Tell me more abou

7 previous questions: What is the keto diet? Why was keto diet originally developed? What is ketosis? What is paleo diet? What do paleo diet and keto diet have in common? How are paleo diet and keto diet different?
7 previous questions: What is the keto diet? Why was keto diet originally developed? What is ketosis? What is paleo diet? What do paleo diet and keto diet have in common? How are paleo diet and keto diet different? What is intermittent fasting?
7 previous questions: Why was keto diet originally developed? What is ketosis? What is paleo diet? What do paleo diet and keto diet have in common? How are paleo diet and keto diet different? What is intermittent fasting? How is intermittent fasting related to keto?
7 previous questions: What is ketosis? What is paleo diet? What do paleo diet and keto diet have in common? How are paleo diet and keto diet different? What is intermittent fasting? How is intermittent fasting related to keto? What is the 16_8 method in intermittent fastin